<a href="https://colab.research.google.com/github/morettitommaso/sql-exercises/blob/main/edb02_librerie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Caricamento dati nel DBMS

La cella qua sotto serve per per scaricare il database e interagire con il DBMS.

Se il DBMS semtte di funzionare, rieseguite la  cella di codice sottostante.

In [1]:
# Per favore, non modificate il codice di questa cella
urldb='https://drive.google.com/uc?export=download&id=1wS2sFxQlw3SycoqNfY5lnSyH0HRYL7i-'
fn='db02librerie.db'
pwd=''

import sqlite3
import urllib
import urllib.request
import zipfile
import os

import pandas as pd
pd.set_option('display.max_colwidth', None) # -1
pd.set_option('display.max_rows', 100)
from IPython.display import display, HTML

def downloadDBFile(urlFile, localFileName, pwd=''):
  # Dowloading the .zip file into the colab remote computation facility

  print('Downloading db file')
  try:
    urllib.urlretrieve(urlFile, localFileName) # Works in Python2
    print('Python2 detected')
  except AttributeError:
    print('Python 3 detected')
    with urllib.request.urlopen(urlFile) as response: # Works in Python3
      data = response.read()
      fd=open(localFileName,'wb')
      fd.write(data)
    fd.close()
  print('Download Done')
  if localFileName.endswith('.zip'):
    zipFileName = localFileName
    localFileName = zipFileName[:-4]
    print('Unzipping the file content')
    with zipfile.ZipFile(zipFileName, 'r') as zip_ref:
      zip_ref.extractall(localFileName)
      print('File %s extracted' % localFileName)
  elif localFileName.endswith('.pez'):
    zipFileName = localFileName
    localFileName = zipFileName[:-4]
    print('Unzipping the password protected file content')
    with zipfile.ZipFile(zipFileName, 'r') as zip_ref:
      #zip_ref.extractall(localFileName, pwd=bytes(pwd,'utf-8')) # it messes up the paths with password encrypted files
      zip_ref.extract(localFileName, pwd=bytes(pwd,'utf-8')) # localFileName: the name of the file in the .zip archive
      print('File %s extracted' % localFileName)

  localFileName = os.path.abspath(localFileName)
  print('File saved in '+localFileName)
  return localFileName
  print('Unzipping the password protected file content')


dbFileFullPath = downloadDBFile(urldb, fn, pwd=pwd)

def rq(query, threshold=100): # rq is for Run Query
    conn = sqlite3.connect(dbFileFullPath)
    if query.replace(' ','').replace('\n','') == '':
      return
    try:
        res = pd.read_sql(query,conn)
        conn.close()
        nrecords=res.shape[0]
        if nrecords<=threshold:
          display(res)
          print("%d records returned" % (nrecords) )
          #HTML("<b> %d records returned </b>" % (nrecords) )
          #return res

        else:
          display(res.iloc[:threshold,:])
          print("Too many records in the result, returned only the first %d" % (threshold))
          #return res.iloc[:threshold,:]

    except (pd.io.sql.DatabaseError, Exception)  as dbe: # (sqlite3.DatabaseError , sqlite3.OperationalError)
        print('Error')
        print(type(dbe))
        print(dbe)

Python 3 detected
Download Done
File saved in /content/db02librerie.db


# EDB02 librerie

In questo esercizio dovrete interagire con un database di nome *librerie* composto da diverse tabelle.

## Schema ER
Qua sotto è riportato lo schema ER del database librerie.

<img src="https://drive.google.com/uc?export=download&id=1JCgOKP3WwSQn7xmBB1DQN1c61zjx59M3" alt="Database ER Schema" width="500">
<!--
https://drive.google.com/file/d/1JCgOKP3WwSQn7xmBB1DQN1c61zjx59M3/view?usp=sharing
-->

[Link](https://drive.google.com/file/d/1JCgOKP3WwSQn7xmBB1DQN1c61zjx59M3/view?usp=sharing) per aprire lo schema in un'altra finestra.

## Schema logico relazionale
Nella figura qua sotto è riportato lo schema relazionale del DB librerie (nota bene: si tratta di uno schema relazionale, non di uno schema Entità Relazioni).


<img src="https://drive.google.com/uc?export=download&id=1yQNp8zqL1423r33lIvSpV0lxASt5UBrr" alt="Database Schema" width="600">
<!-- https://www.dropbox.com/s/r115zp5k2k2zv8l/db02librerieSchema.png?dl=1 -->

Agli estremi di ogni freccia, sono presenti 2 simboli (uno per la cardinalità minima, uno per la cardinalità massima), ogni simbolo può essere un elemento del seguente alfabeto (tra parentesi i significati):

* O (zero)
* | (uno)
* <img src="https://docs.google.com/drawings/d/e/2PACX-1vRbanf6cNiZWPXTDatkH-xNadoI8ZKGuLq2au4GYB1QkrXH528yClegyf5G1pFU6vy5TnlPKQ1ti1dj/pub?w=360&amp;h=20"><!-- &#5833; --> (molti)

Es. <img src="https://docs.google.com/drawings/d/e/2PACX-1vTYY7uTwq8rVX7ZhZJnKerDeICjkEkPWnJDacJO7XnAUZjGQEapRHxjqiKr8ui4QHudPZtQacXI7aCm/pub?w=41&amp;h=27"> viene interpretato come [1:N] (minima 1, massima N).

Il modo di leggere le numerosità in questi schemi relazionali è invertita rispetto agli schemi ER.

[Link](https://drive.google.com/file/d/1yQNp8zqL1423r33lIvSpV0lxASt5UBrr/view?usp=sharing) per aprire lo schema in un'altra finestra.



# Esercizi

Svolgete gli esercizi indicati nelle celle sottostanti.

**Riepilogo sintassi query SQL**

SELECT Attributo1, Attributo2, ..., *

FROM Tabella1, Tabella2, …, TabellaN

WHERE CondizioniSemplici

ORDER BY ListaAttributiDiOrdinamento

;


## (EDB02) 1

Ispezionate le tabelle del database librerie e cercate di farvi un’idea del contenuto del database.

Per ispezionare una tabella, potete utilizzare la query generica:

SELECT * FROM  scrivi_qui_il_nome_della_tabella;

# Per i nomi delle tabelle fate riferimento allo schema della figura precedente.


In [7]:
rq("""
select * from autori;
""")

# Potete aggiungere altre caselle di codice qua sotto per svolgere altre query

,codice_autore,cognome,nome
0,1,Archer,Jeffrey
1,2,Christie,Agatha
2,3,Clarke,Arthur C.
3,4,Francis,Dick
4,5,Cussler,Clive
5,6,King,Stephen
6,7,Pratt,Philip
7,8,Adamski,Joseph
8,10,Harmon,Willis
9,11,Rheingold,Howard


22 records returned


In [8]:
rq("""
select * from scorte;
""")


,codice_libro,codice_negozio,num_copie_disponibili
0,0180,1,2
1,0189,2,2
2,0200,1,1
3,0200,2,3
4,079X,2,1
5,079X,3,2
6,079X,4,3
7,1351,1,1
8,1351,2,4
9,1351,3,2


34 records returned


## (EDB02) 2

Scrivete su un foglio di carta il nome delle tabelle principali, per ogni tabella individuate (se presenti):

* la chiave primaria (o le chiavi primarie se più attributi sono coinvolti)

* i collegamenti con le altre tabelle (individuate cioè le chiavi secondarie). Suggerimento: tracciate delle frecce per collegare le chiavi esterne con le corrispondenti chiavi primarie

* nello schema sono presenti 2 tabelle ponte. Quali sono? (Se non sapete che cosa è una tabella ponte, ignorate questo punto)

Questo foglio sarà utilizzato negli esercizi successivi.

NB: scrivere le tabelle e le chiavi su carta, vi sarà molto utile come riferimento per svolgere gli esercizi indicati di seguito, dedicate un tempo congruo ma senza esagerare per svolgere questo punto (NON più di 10 minuti).


*   AUTORI(PK:cod_autore)
*   NEGOZI(PK:cod_negozio)
*   LIBRI(PK:codice, FK:cod_autore,cod_formato)
*   EDITORI(PK:codice)
*   FORMATO(PK:codice)

ponti:
*   HASCELTO(FK:cod_libro, cod_autore)
*   SCORTE(FK:cod_libro, cod_negozio)





## (EDB02) 3

Visualizzate il contenuto della tabella libri

In [10]:
rq("""
select * from LIBRI;
""")

,codice,titolo,codice_editore,tipo,prezzo,cod_format
0,0180,Shyness,BB,PSY,7.65,1
1,0189,Kane and Abel,PB,FIC,5.55,2
2,0200,Stranger,BB,FIC,8.75,3
3,0378,Dunwich Horror and Others,PB,HOR,19.75,1
4,079X,Smokescreen,PB,MYS,4.55,3
5,0808,Knockdown,PB,MYS,4.75,3
6,1351,Cujo,SI,HOR,6.65,1
7,1382,Marcel Duchamp,PB,ART,11.25,2
8,138X,Death on the Nil,BB,MYS,3.95,1
9,2226,Ghost from the Grand Banks,BB,SFI,19.95,3


29 records returned


## (EDB02) 4

Visualizzate solo il contenuto degli attributi titolo, tipo, prezzo della tabella libri.

In [11]:
rq("""
SELECT titolo,tipo,prezzo
FROM LIBRI;
""")

,titolo,tipo,prezzo
0,Shyness,PSY,7.65
1,Kane and Abel,FIC,5.55
2,Stranger,FIC,8.75
3,Dunwich Horror and Others,HOR,19.75
4,Smokescreen,MYS,4.55
5,Knockdown,MYS,4.75
6,Cujo,HOR,6.65
7,Marcel Duchamp,ART,11.25
8,Death on the Nil,MYS,3.95
9,Ghost from the Grand Banks,SFI,19.95


29 records returned


## (EDB02) 5

Visualizzate titolo e prezzo dei libri che costano meno di 10 euro.

Nota: 17 tuple nel risultato

In [12]:
rq("""
SELECT titolo,prezzo
FROM LIBRI
WHERE libri.prezzo < 10;
""")

,titolo,prezzo
0,Shyness,7.65
1,Kane and Abel,5.55
2,Stranger,8.75
3,Smokescreen,4.55
4,Knockdown,4.75
5,Cujo,6.65
6,Death on the Nil,3.95
7,Prodigal Daughter,5.45
8,Hymns to the Night,6.75
9,Higher Creativity,9.75


17 records returned


## (EDB02) 6

Ora estrarremo alcune informazioni sulle tipologie di libri.

#### (EDB02) 6.a

Visualizzate le informazioni dei libri di informatica (i libri con tipo uguale a 'CS'), CS è l'acronimo di "computer science".

Nota: 4 tuple nel risultato


In [13]:
rq("""
select *
from libri
where libri.tipo == "CS";
""")

,codice,titolo,codice_editore,tipo,prezzo,cod_format
0,5790,Database Systems,BF,CS,54.95,2
1,669X,A Guide to SQL,BF,CS,23.95,1
2,6908,DOS Essentials,BF,CS,20.50,2
3,7947,dBASE Programming,BF,CS,39.90,1


4 records returned


#### (EDB02) 6.b

Visualizzate le informazioni dei libri di *science fiction* (i libri con tipo uguale a 'FIC'), FIC è usato per indicare *Science Fiction*.

Nota: 6 tuple nel risultato

In [14]:
rq("""
select *
from libri
where libri.tipo == "FIC";
""")

,codice,titolo,codice_editore,tipo,prezzo,cod_format
0,0189,Kane and Abel,PB,FIC,5.55,2
1,0200,Stranger,BB,FIC,8.75,3
2,2766,Prodigal Daughter,PB,FIC,5.45,1
3,3743,First Among Equals,PB,FIC,3.95,2
4,8720,Castle,BB,FIC,12.15,2
5,9611,Amerika,BB,FIC,10.95,3


6 records returned


#### (EDB02) 6.c

Visualizzate con un’unica query le informazioni sia dei libri di *informatica* sia dei libri di *science fiction* (tipi: rispettivamente 'CS' e 'FIC').

Nota: 10 tuple di risultato.


In [17]:
rq("""
select *
from libri
where (libri.tipo == "FIC" or libri.tipo == "CS");
""")

,codice,titolo,codice_editore,tipo,prezzo,cod_format
0,0189,Kane and Abel,PB,FIC,5.55,2
1,0200,Stranger,BB,FIC,8.75,3
2,2766,Prodigal Daughter,PB,FIC,5.45,1
3,3743,First Among Equals,PB,FIC,3.95,2
4,5790,Database Systems,BF,CS,54.95,2
5,669X,A Guide to SQL,BF,CS,23.95,1
6,6908,DOS Essentials,BF,CS,20.50,2
7,7947,dBASE Programming,BF,CS,39.90,1
8,8720,Castle,BB,FIC,12.15,2
9,9611,Amerika,BB,FIC,10.95,3


10 records returned


#### (EDB02) 6.d

Se nella query precedente avete usato l’operatore OR, provate ad usare l’operatore AND. Se invece avete usato l’AND provate con l’OR. Cosa cambia? Qual è il motivo? Suggerimento: se a questo punto avete dei dubbi (anche minimi), chiamate il docente/tutor che trovate in aula.

In [18]:
rq("""
select *
from libri
where (libri.tipo == "FIC" and libri.tipo == "CS");
""")

,codice,titolo,codice_editore,tipo,prezzo,cod_format


0 records returned


## (EDB02) 7

Ora andremo a scrivere delle query in cui dovremo mettere in join più tabelle.

Per svolgere questo esercizio dovete aver sottomano il foglio che avete annotato/creato nell'esercizio 2.

Se vi sentite molto ferrati sull’argomento potete saltare direttamente al *sottopunto g*, altrimenti svolgete tutti i sottopunti seguenti.

#### (EDB02) 7.a

Con una query, visualizzate dalla tabella libri i dati del libro dal titolo "Shyness". A seconda del DBMS che state usando, il contenuto delle stringhe può essere *case sensitive* oppure no. I comandi SQL sono invece sempre *case insensitive*.

Nota: 1 record nel risultato.

In [19]:
rq("""
select *
from LIBRI
where libri.titolo == "Shyness";
""")

,codice,titolo,codice_editore,tipo,prezzo,cod_format
0,0180,Shyness,BB,PSY,7.65,1


1 records returned


#### (EDB02) 7.b

Nella tabella libri, qual è la *foreign key* che  permette di collegare un libro al suo editore?

Per il libro dal titolo Shyness, qual è il valore assunto dalla foreign key?

Scrivete la vostra risposta in questa cella (doppio click per entrare in modalità editing)

BB




#### (EDB02) 7.c

(Con una query) visualizzate il contenuto della tabella editori.

In [20]:
rq("""
select *
from editori
""")

,codice,nome,citta,stato
0,AH,Arkham House Publ.,Sauk City,WI
1,AP,Arcade Publishing,New York,NY
2,AW,Addison Wesley,Reading,MA
3,BB,Bantam Books,New York,NY
4,BF,Best and Furrow,Boston,MA
5,JT,Jeremy P. Tarcher,Los Angeles,CA
6,MP,McPherson and Co.,Kinston,NY
7,PB,Pocket Books,New York,NY
8,RH,Random House,New York,NY
9,RZ,Rizzoli,New York,NY


14 records returned


#### (EDB02) 7.d

Rispondete alle prossime 2 domande guardando il risultato della query precedente (non dovete scrivere altre query).

Qual è l'attributo che funge da chiave della tabella editori (a cui fa riferimento la chiave straniera presente nella tabella libri)?

CODICE

Qual è l'editore che edita il libro Shyness?

BB

Scrivete la vostra risposta in questa cella (doppio click per entrare in modalità editing)


#### (EDB02) 7.e

Eseguite la query

select * from libri, editori;

In [22]:
rq("""
select * from libri, editori;
""")

,codice,titolo,codice_editore,tipo,prezzo,cod_format,codice,nome,citta,stato
0,0180,Shyness,BB,PSY,7.65,1,AH,Arkham House Publ.,Sauk City,WI
1,0180,Shyness,BB,PSY,7.65,1,AP,Arcade Publishing,New York,NY
2,0180,Shyness,BB,PSY,7.65,1,AW,Addison Wesley,Reading,MA
3,0180,Shyness,BB,PSY,7.65,1,BB,Bantam Books,New York,NY
4,0180,Shyness,BB,PSY,7.65,1,BF,Best and Furrow,Boston,MA
5,0180,Shyness,BB,PSY,7.65,1,JT,Jeremy P. Tarcher,Los Angeles,CA
6,0180,Shyness,BB,PSY,7.65,1,MP,McPherson and Co.,Kinston,NY
7,0180,Shyness,BB,PSY,7.65,1,PB,Pocket Books,New York,NY
8,0180,Shyness,BB,PSY,7.65,1,RH,Random House,New York,NY
9,0180,Shyness,BB,PSY,7.65,1,RZ,Rizzoli,New York,NY


Too many records in the result, returned only the first 100


#### (EDB02) 7.f

Nel risultato della query precedente, guardate i record che fanno riferimento al libro Shyness *Shyness*. Il libro è stato associato ad editori in maniera errata? (Non serve scrivere la risposta).

Vi ricordate come dal prodotto cartesiano si passa ad un join? Se non ve lo ricordate, andate a ripassare la parte di teoria corrispondente.


#### (EDB02) 7.g

Visualizzate per ogni libro, il titolo del libro e il nome dell'editore che lo edita (dovete effettuare il join tra la tabella libri e la tabella editori).

Nota: 29 tuple nel risultato

In [24]:
rq("""
select *
from libri, editori
where libri.codice_editore == editori.codice;
""")

,codice,titolo,codice_editore,tipo,prezzo,cod_format,codice,nome,citta,stato
0,0180,Shyness,BB,PSY,7.65,1,BB,Bantam Books,New York,NY
1,0189,Kane and Abel,PB,FIC,5.55,2,PB,Pocket Books,New York,NY
2,0200,Stranger,BB,FIC,8.75,3,BB,Bantam Books,New York,NY
3,0378,Dunwich Horror and Others,PB,HOR,19.75,1,PB,Pocket Books,New York,NY
4,079X,Smokescreen,PB,MYS,4.55,3,PB,Pocket Books,New York,NY
5,0808,Knockdown,PB,MYS,4.75,3,PB,Pocket Books,New York,NY
6,1351,Cujo,SI,HOR,6.65,1,SI,Signet,New York,NY
7,1382,Marcel Duchamp,PB,ART,11.25,2,PB,Pocket Books,New York,NY
8,138X,Death on the Nil,BB,MYS,3.95,1,BB,Bantam Books,New York,NY
9,2226,Ghost from the Grand Banks,BB,SFI,19.95,3,BB,Bantam Books,New York,NY


29 records returned


## (EDB02) 8

Con una query, visualizzate per ogni libro il titolo, il prezzo e la descrizione del formato (vedi tabella formato).

Nota: 29 tuple nel risultato

In [25]:
rq("""
select titolo, prezzo, descrizione
from libri, formato
where libri.cod_format == formato.codice;
""")

,titolo,prezzo,descrizione
0,Shyness,7.65,Tascabile
1,Kane and Abel,5.55,Paperback
2,Stranger,8.75,Copertina rigida
3,Dunwich Horror and Others,19.75,Tascabile
4,Smokescreen,4.55,Copertina rigida
5,Knockdown,4.75,Copertina rigida
6,Cujo,6.65,Tascabile
7,Marcel Duchamp,11.25,Paperback
8,Death on the Nil,3.95,Tascabile
9,Ghost from the Grand Banks,19.95,Copertina rigida


29 records returned


## (EDB02) 9

Ora ci occuperemo degli autori dei libri

#### (EDB02) 9.a

Per visualizzare con un'unica query il titolo del libro e il cognome dell'autore che lo ha scritto, quante tabelle dovete mettere in join? Quali tabelle? Individuate le *foreign keys* e le chiavi (primarie) a cui fanno riferimento.

Scrivete la vostra risposta in questa cella (doppio click per entrare in modalità editing)

#### (EDB02) 9.b

Visualizzate con un’unica query il titolo del libro e il cognome dell'autore che lo ha scritto.

Nota: 33 tuple nel risultato.

In [33]:
rq("""
select titolo, cognome
from autori, hascritto, libri
where (libri.codice == hascritto.codice_libro)
and (autori.codice_autore == hascritto.codice_autore);
""")

,titolo,cognome
0,Kane and Abel,Archer
1,Prodigal Daughter,Archer
2,First Among Equals,Archer
3,Death on the Nil,Christie
4,Evil Under the Sun,Christie
5,Ghost from the Grand Banks,Clarke
6,Smokescreen,Francis
7,Knockdown,Francis
8,Risk,Francis
9,Vortex,Cussler


33 records returned


#### (EDB02) 9.c

Nella query precedente, secondo voi, perché ci sono 33 tuple nel risultato e non 29 (il numero di record presenti nella tabella libri)?

Scrivete la vostra risposta in questa cella (doppio click per entrare in modalità editing)

###alcuni libri hanno più autori? (lo controllo nelle prossime cella)

In [29]:
rq("""
select * from libri;
""")

,codice,titolo,codice_editore,tipo,prezzo,cod_format
0,0180,Shyness,BB,PSY,7.65,1
1,0189,Kane and Abel,PB,FIC,5.55,2
2,0200,Stranger,BB,FIC,8.75,3
3,0378,Dunwich Horror and Others,PB,HOR,19.75,1
4,079X,Smokescreen,PB,MYS,4.55,3
5,0808,Knockdown,PB,MYS,4.75,3
6,1351,Cujo,SI,HOR,6.65,1
7,1382,Marcel Duchamp,PB,ART,11.25,2
8,138X,Death on the Nil,BB,MYS,3.95,1
9,2226,Ghost from the Grand Banks,BB,SFI,19.95,3


29 records returned


In [27]:
rq("""
select cognome, codice_libro
from autori, hascritto
where (autori.codice_autore == hascritto.codice_autore);
""")

,cognome,codice_libro
0,Archer,0189
1,Archer,2766
2,Archer,3743
3,Christie,138X
4,Christie,6128
5,Clarke,2226
6,Francis,079X
7,Francis,0808
8,Francis,7559
9,Cussler,3906


33 records returned


## (EDB02) 10

Come il punto precedente (... il titolo del libro e il cognome dell'autore che lo ha scritto), ma visualizzate solamente i dati dei libri scritti da Kafka.

Nota: 2 tuple nel risultato

In [34]:
rq("""
select titolo, cognome
from autori, hascritto, libri
where ((libri.codice == hascritto.codice_libro)
and (autori.codice_autore == hascritto.codice_autore))
and autori.cognome = "Kafka";
""")

,titolo,cognome
0,Castle,Kafka
1,Amerika,Kafka


2 records returned


## (EDB02) 11

Come il punto precedente (... il titolo del libro e il cognome dell'autore che lo ha scritto), ma visualizzate solamente i libri scritti da Kafka o da Agata Cristie.

Nota: 4 tuple nel risultato

ps: controllate nel database come sono stati scritti i nomi e cognomi dei due autori :-)

In [35]:
rq("""
select titolo, cognome
from autori, hascritto, libri
where ((libri.codice == hascritto.codice_libro)
and (autori.codice_autore == hascritto.codice_autore))
and (autori.cognome == "Kafka" or autori.cognome == "Christie");
""")

,titolo,cognome
0,Death on the Nil,Christie
1,Evil Under the Sun,Christie
2,Castle,Kafka
3,Amerika,Kafka


4 records returned


## (EDB02) 12
Visualizzate il nome e la sede delle librerie dove è in vendita il libro 'dBASE Programming'.

Nota: 1 tupla nel risultato

In [38]:
rq("""
select nome, sede
from negozi, scorte, libri
where (negozi.codice_negozio == scorte.codice_negozio)
and (libri.codice == scorte.codice_libro)
and (libri.titolo == 'dBASE Programming');
""")

,nome,sede
0,Libreria Rossi,viale Sarca 5


1 records returned
